una vez que tenemos los datos de las fuentes, en este caso ya tenemos el notebook procederemos a examinarlo

## Estudio
Para ver que hay en el dataframe vamos a estudiar las variables que tenemos y la informacion que podemos sacar. 
El df tiene visos de pertencer a una empresa que recoge las quejas de los cleintes de varias compañias y las gestiona envandolas a las empresas. posterioemente, no siempre, recoge el resultado de las quejas y si estas han derivado posteriormente en problemas mas halla(reclamaciones ante la autoridad, denuncias etc.)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import regex as re
import skrub
import fuentes


In [2]:
#!pip install skrub

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
import fuentes
main_df=fuentes.main_df()
zip_code=pd.read_csv("../data/zip_code.csv")


In [5]:
skrub.TableReport(main_df)

Processing column  14 / 14


,,,,,,,,,,,,,,


en este dataframe vemos que hay varias filas en las cuales la respuesta de la compañia esta en tiempo pero no se ha dado todavia, en estas situaciones no tenemos informacion adicional sobre si definitivamente se ha escalado la situacion o no. ello hace que estas filas  no sean realmente informativas.

In [6]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28156 entries, 0 to 28155
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            28156 non-null  int64  
 1   Complaint ID          28156 non-null  int64  
 2   Product               28156 non-null  object 
 3   Sub-product           17582 non-null  object 
 4   Issue                 28154 non-null  object 
 5   Sub-issue             13211 non-null  object 
 6   State                 27735 non-null  object 
 7   ZIP code              27876 non-null  float64
 8   Date received         28156 non-null  object 
 9   Date sent to company  28156 non-null  object 
 10  Company               28156 non-null  object 
 11  Company response      28156 non-null  object 
 12  Timely response?      28156 non-null  object 
 13  Consumer disputed?    6006 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 3.0+ MB


In [7]:
for x in main_df.columns[2:]:
    print(x,((main_df[x].isnull().sum()/28156)*100))



Product 0.0
Sub-product 37.55505043330019
Issue 0.007103281716152862
Sub-issue 53.079272623952264
State 1.4952408012501777
ZIP code 0.9944594402614008
Date received 0.0
Date sent to company 0.0
Company 0.0
Company response 0.0
Timely response? 0.0
Consumer disputed? 78.66884500639296


In [8]:
temp=main_df[main_df["Consumer disputed?"].notnull()]
temp["Consumer disputed?"].value_counts()

Consumer disputed?
Yes    4708
No     1298
Name: count, dtype: int64

In [9]:
pd.set_option('display.max_rows', 1000)
display(temp["Company"].value_counts())
len(temp["Company"])

Company
Equifax                                                  524
JPMorgan Chase                                           318
Wells Fargo                                              317
TransUnion                                               300
Bank of America                                          296
Experian                                                 269
Citibank                                                 245
Nationstar Mortgage                                      216
Ocwen                                                    198
Enhanced Recovery Company, LLC                           126
GE Capital Retail                                        116
U.S. Bancorp                                             108
Discover                                                  87
Amex                                                      83
Encore Capital Group                                      79
Capital One                                               78
Green Tree Servi

6006

In [10]:
df_fechas=pd.to_datetime(main_df["Date sent to company"])-pd.to_datetime(main_df["Date received"])

#main_df["Date received"] = pd.to_datetime(main_df["Date received"])

In [11]:
# el estudio de las fechas a demostrado que solamente existe 1 año en el dataframe
# por tanto, no sera relevante a la hora de trabajar el dataframe. 
años=pd.to_datetime(main_df["Date received"],yearfirst=True)

años.dt.year

columna_años=años.dt.year

columna_años.unique()


array([2015], dtype=int32)

In [12]:
main_df.describe()

,Unnamed: 0,Complaint ID,ZIP code
count,28156.000000,2.815600e+04,27876.000000
mean,14077.500000,1.225267e+06,49921.466423
std,8128.081426,2.835272e+04,30633.853920
min,0.000000,1.177839e+06,9.000000
25%,7038.750000,1.200523e+06,23228.000000
50%,14077.500000,1.224512e+06,44111.000000
75%,21116.250000,1.248698e+06,78744.000000
max,28155.000000,1.292165e+06,99999.000000


In [13]:
main_df["Timely response?"].value_counts()
# aqui de detecta un problema de balance de los datos.

Timely response?
Yes    27444
No       712
Name: count, dtype: int64

In [14]:
main_df["Consumer disputed?"].value_counts()
# aqui tambien tenemos un problema de balanceo en la variable objetivo pero no es tan mala como en 
# en la del tiempo de respuesta

Consumer disputed?
Yes    4708
No     1298
Name: count, dtype: int64

In [15]:
# vamos a ir viendo las columnas para determinar que hacer con ellas.
# emprezamos con la variable estado, "state". la variable no tienen muchos nan pero eso no implica que 
# los que esten no sean un incordio para operar. 
#main_df[pd.isnull(main_df["State"])]


In [16]:
'''zip_code=pd.read_csv("../data/zip_code.csv")
state=[]
zip=[]
for i in zip_code.values:
    resultado = re.findall(r"[A-Z]{2}", i[0])
    state.append(resultado[0])
    x=i[2].replace("to","<z<")
    zip.append(x)

zip_code=pd.DataFrame({'State': state, 'ZIP Codes': zip})
'''

'zip_code=pd.read_csv("../data/zip_code.csv")\nstate=[]\nzip=[]\nfor i in zip_code.values:\n    resultado = re.findall(r"[A-Z]{2}", i[0])\n    state.append(resultado[0])\n    x=i[2].replace("to","<z<")\n    zip.append(x)\n\nzip_code=pd.DataFrame({\'State\': state, \'ZIP Codes\': zip})\n'

In [17]:
# para hacer la operaciones utilizaremos una copia del cataframe original, al que todavia no hemos modificado.
main_df=fuentes.main_df()
df=main_df.copy()

In [18]:
df=main_df.copy()
df["Date received"]=df_fechas
print(df.columns)

df.drop('Date sent to company',inplace=True,axis=1)

Index(['Unnamed: 0', 'Complaint ID', 'Product', 'Sub-product', 'Issue',
       'Sub-issue', 'State', 'ZIP code', 'Date received',
       'Date sent to company', 'Company', 'Company response',
       'Timely response?', 'Consumer disputed?'],
      dtype='object')


In [19]:
df

,Unnamed: 0,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Date received,Company,Company response,Timely response?,Consumer disputed?
0,0,1291006,Debt collection,NaN,Communication tactics,Frequent or repeated calls,TX,76119.0,0 days,"Premium Asset Services, LLC",In progress,Yes,NaN
1,1,1290580,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,TX,77479.0,0 days,Accounts Receivable Consultants Inc.,Closed with explanation,Yes,NaN
2,2,1290564,Mortgage,FHA mortgage,"Application, originator, mortgage broker",NaN,MA,2127.0,0 days,RBS Citizens,Closed with explanation,Yes,Yes
3,3,1291615,Credit card,NaN,Other,NaN,CA,92592.0,0 days,Navy FCU,In progress,Yes,NaN
4,4,1292165,Debt collection,Non-federal student loan,Cont'd attempts collect debt not owed,Debt resulted from identity theft,NaN,43068.0,0 days,Transworld Systems Inc.,In progress,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28151,28151,1178184,Debt collection,NaN,Cont'd attempts collect debt not owed,Debt is not mine,TX,75201.0,1 days,"Enhanced Recovery Company, LLC",Closed with non-monetary relief,Yes,No
28152,28152,1178183,Consumer loan,Vehicle loan,Taking out the loan or lease,NaN,GA,30043.0,6 days,DriveTime,Closed with explanation,Yes,NaN
28153,28153,1178182,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,MN,55105.0,1 days,Regions,Closed with non-monetary relief,Yes,No
28154,28154,1178181,Payday loan,NaN,Can't stop charges to bank account,NaN,CA,92509.0,6 days,Moneytree Inc,Closed with explanation,Yes,NaN


In [20]:
from skrub import TableReport

TableReport(df)

Processing column  13 / 13


,,,,,,,,,,,,,


In [21]:
# arreglamos las columnas de Isuue y la de timely response.
listatemp=[]
for i in df["Timely response?"].values:
    if i == "Yes":
        listatemp.append(True)
    else:
        listatemp.append(False)
listatemp=pd.Series(listatemp)
df["Timely response?"]=listatemp.astype(bool)

In [22]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [23]:
df.drop([11730, 13198], inplace=True)

In [24]:
df.drop("Sub-issue",axis=1,inplace=True)

Tras el tratamiento de algunos de los problemas, vamos a rellenar los valores usando el KNN ya hemos eliminado algunas de las columnas que nos podrian molestar, como aquella que tien mas de un 50% de valores faltantes. 

In [25]:
df.columns

Index(['Complaint ID', 'Product', 'Sub-product', 'Issue', 'State', 'ZIP code',
       'Date received', 'Company', 'Company response', 'Timely response?',
       'Consumer disputed?'],
      dtype='object')

In [26]:
# antes de proceder con esto debemos de pasar a numero todas las categorias para que el progrma los pueda procesar. 
# por tanto, procederemos con el label encoding. etc. 
'''from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
imputer.fit(df[['Complaint ID', 'Product', 'Sub-product', 'Issue', 'State', 'ZIP code',
       'Date received', 'Company', 'Company response', 'Timely response?']])
df[['Complaint ID', 'Product', 'Sub-product', 'Issue', 'State', 'ZIP code',
       'Date received', 'Company', 'Company response', 'Timely response?']] = imputer.transform(df[['Complaint ID', 'Product', 'Sub-product', 'Issue', 'State', 'ZIP code',
       'Date received', 'Company', 'Company response', 'Timely response?']])'''

"from sklearn.impute import KNNImputer\n\nimputer = KNNImputer(n_neighbors=2)\nimputer.fit(df[['Complaint ID', 'Product', 'Sub-product', 'Issue', 'State', 'ZIP code',\n       'Date received', 'Company', 'Company response', 'Timely response?']])\ndf[['Complaint ID', 'Product', 'Sub-product', 'Issue', 'State', 'ZIP code',\n       'Date received', 'Company', 'Company response', 'Timely response?']] = imputer.transform(df[['Complaint ID', 'Product', 'Sub-product', 'Issue', 'State', 'ZIP code',\n       'Date received', 'Company', 'Company response', 'Timely response?']])"

In [27]:
# procederemos ahora a trabajar los estados y el Zip code.
TableReport(df)

Processing column  11 / 11


,,,,,,,,,,,


In [28]:
columnas_a_cambiar=df[df.columns[4:6]]
print(columnas_a_cambiar)

      State  ZIP code
0        TX   76119.0
1        TX   77479.0
2        MA    2127.0
3        CA   92592.0
4       NaN   43068.0
...     ...       ...
28151    TX   75201.0
28152    GA   30043.0
28153    MN   55105.0
28154    CA   92509.0
28155    OH   45068.0

[28154 rows x 2 columns]


In [29]:
zip_code

zip_code=pd.read_csv("../data/zip_code.csv")
state=[]
zip=[]
for i in zip_code.values:
    resultado = re.findall(r"[A-Z]{2}", i[0])
    state.append(resultado[0])
    x=i[2].replace("to","< z <")
    zip.append(x)

zip_code=pd.DataFrame({'State': state, 'ZIP Codes': zip})

In [30]:
display(zip_code,columnas_a_cambiar)

,State,ZIP Codes
0,AL,35004 < z < 36925
1,AK,99501 < z < 99950
2,AZ,85001 < z < 86556
3,AR,71601 < z < 72959
4,CA,90001 < z < 96162
5,CO,80001 < z < 81658
6,CT,6001 < z < 6928
7,DE,19701 < z < 19980
8,FL,32003 < z < 34997
9,GA,30002 < z < 39901


,State,ZIP code
0,TX,76119.0
1,TX,77479.0
2,MA,2127.0
3,CA,92592.0
4,NaN,43068.0
...,...,...
28151,TX,75201.0
28152,GA,30043.0
28153,MN,55105.0
28154,CA,92509.0


In [31]:
listado_estado=[]
for i in columnas_a_cambiar.values:
    if pd.isna(i[0]) and not pd.isna(i[1]):
        encontrado=False
        for x in zip_code.values:
            z=i[1]
            y=x[1]
            if eval(y):
                listado_estado.append(x[0])
                encontrado=True
                break
        if not encontrado:
            listado_estado.append(np.nan)
    else:
        listado_estado.append(i[0])


In [32]:
display(listado_estado)
print(len(listado_estado),listado_estado.count(np.nan))

['TX',
 'TX',
 'MA',
 'CA',
 'OH',
 'OH',
 'OH',
 'CA',
 'NJ',
 'TX',
 'ND',
 'RI',
 'CO',
 'TX',
 'UT',
 'AL',
 'PA',
 'TX',
 'CA',
 'NY',
 'NC',
 'GA',
 'IL',
 'CA',
 'WI',
 'IL',
 'MI',
 'FL',
 'FL',
 'CA',
 'CT',
 'OR',
 'GA',
 'CA',
 'VA',
 'WA',
 'WA',
 'PA',
 'TN',
 'AL',
 'MI',
 'WI',
 'PA',
 'IL',
 'MD',
 'FL',
 'IA',
 'CA',
 'MI',
 'FL',
 'GA',
 'KY',
 'MI',
 'NY',
 'FL',
 'WI',
 'CA',
 'LA',
 'NC',
 'IL',
 'CO',
 'OK',
 'MA',
 'IL',
 'TN',
 'NE',
 'IA',
 'KS',
 'MO',
 'TX',
 'OR',
 'NH',
 'OH',
 'CO',
 'MD',
 'WA',
 'CA',
 'CA',
 'IN',
 'CA',
 'NC',
 'OH',
 'CA',
 'FL',
 'ND',
 'CA',
 'FL',
 'GA',
 'FL',
 'MI',
 'WA',
 'CA',
 'PA',
 'OH',
 'NJ',
 'PA',
 'DC',
 'NC',
 'NH',
 'FL',
 'NV',
 'TX',
 'AL',
 'MI',
 'NC',
 'CA',
 'OK',
 'KY',
 'FL',
 'KY',
 'MD',
 'CA',
 'GA',
 'MD',
 'NJ',
 'WA',
 'FL',
 'NY',
 'WI',
 'TX',
 'MO',
 'NY',
 'TX',
 'TX',
 'IL',
 'CA',
 'MI',
 'TX',
 'NC',
 'TX',
 'PA',
 'TN',
 'NY',
 'ME',
 'TN',
 'CA',
 'MO',
 'TX',
 'IN',
 'CT',
 'TX',
 'NJ',
 'PA',

28154 167


In [33]:
df["State"]=listado_estado

In [34]:
import random
listado_zip=[]
for i in columnas_a_cambiar.values:
    if pd.isna(i[1]) and not pd.isna(i[0]):
        encontrado=False
        for x in zip_code.values:

               


            if x[0]==i[0]:
                h=x[1]
                h=h.replace("z"," ")
                h=h.split("<")
                listatemp=[]
                n1=int(h[0].replace(" ",""))
                n2=int(h[-1].replace(" ",""))

                
                listado_zip.append(random.randint(n1,n2))
                encontrado=True
                break
        if not encontrado:
            listado_zip.append(np.nan)
    else:
        listado_zip.append(i[1])

In [ ]:
listado_zip.

[76119.0,
 77479.0,
 2127.0,
 92592.0,
 43068.0,
 43068.0,
 44241.0,
 95124.0,
 7067.0,
 77433.0,
 58237.0,
 2893.0,
 80301.0,
 78205.0,
 84405.0,
 36756.0,
 19140.0,
 77099.0,
 92069.0,
 10470.0,
 28110.0,
 30058.0,
 60411.0,
 92592.0,
 53183.0,
 60628.0,
 48091.0,
 33990.0,
 32713.0,
 95032.0,
 6611.0,
 97233.0,
 30518.0,
 91205.0,
 22180.0,
 98502.0,
 98502.0,
 19149.0,
 38305.0,
 35115.0,
 48334.0,
 53579.0,
 19406.0,
 61554.0,
 21704.0,
 33063.0,
 51526.0,
 94610.0,
 48088.0,
 33462.0,
 30075.0,
 42223.0,
 48473.0,
 11415.0,
 34473.0,
 53105.0,
 90801.0,
 70774.0,
 27205.0,
 61032.0,
 80540.0,
 73072.0,
 2038.0,
 62704.0,
 37074.0,
 68135.0,
 50317.0,
 66606.0,
 63040.0,
 76205.0,
 97070.0,
 3753.0,
 44121.0,
 80516.0,
 21085.0,
 98311.0,
 91355.0,
 90280.0,
 46307.0,
 92844.0,
 28365.0,
 43204.0,
 90201.0,
 33179.0,
 58104.0,
 95132.0,
 33179.0,
 30093.0,
 33154.0,
 48237.0,
 98115.0,
 94709.0,
 19064.0,
 43702.0,
 8059.0,
 18102.0,
 20008.0,
 27707.0,
 3290.0,
 32086.0,
 89148.0

In [41]:
print(len(listado_zip),listado_zip.count(np.nan))



28154 0


In [37]:
df["ZIP code"]=listado_zip

In [40]:
TableReport(df)

Processing column  11 / 11


,,,,,,,,,,,


In [39]:
# vamos a borrar las lineas que tienen valores faltantes en state y Zip code
#df[df["State"].isna()]
#df=df[df["State"].isna()]
#df.drop(df[df["State"].isna()],axis=0)

df = df.dropna(subset=['State'])

In [ ]:
# una vez tratados los datos faltantes de algunas de las columnas vamos a probar con rellenar los datos faltantes en la ultima columna y en la segunda columna
# esto lo haremos primero realizando un encodign (labelencoding) y, posteriormente lo aplicaremos al set. 
